In [3]:
import os
from dotenv import load_dotenv
from pinecone import Pinecone

# .env 로드
load_dotenv(override=True)

# 환경변수 읽기
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

if not PINECONE_API_KEY:
    raise ValueError("❌ PINECONE_API_KEY가 .env에 없습니다")

# Pinecone 연결
pc = Pinecone(api_key=PINECONE_API_KEY)

# 인덱스 확인
index = pc.Index("realestate2")
print(index.describe_index_stats())

{'_response_info': {'raw_headers': {'connection': 'keep-alive',
                                    'content-length': '151',
                                    'content-type': 'application/json',
                                    'date': 'Mon, 19 Jan 2026 05:42:43 GMT',
                                    'grpc-status': '0',
                                    'server': 'envoy',
                                    'x-envoy-upstream-service-time': '37',
                                    'x-pinecone-request-id': '7881142262978464916',
                                    'x-pinecone-request-latency-ms': '36',
                                    'x-pinecone-response-duration-ms': '38'}},
 'dimension': 3072,
 'index_fullness': 0.0,
 'memoryFullness': 0.0,
 'metric': 'cosine',
 'namespaces': {},
 'storageFullness': 0.0,
 'total_vector_count': 0,
 'vector_type': 'dense'}


In [4]:
import os
import re
import uuid
import pandas as pd
from dotenv import load_dotenv
from openai import OpenAI
from pinecone import Pinecone

# =========================
# 0) ENV & CLIENT
# =========================
load_dotenv(override=True)

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

if not OPENAI_API_KEY or not PINECONE_API_KEY:
    raise ValueError("❌ .env에 OPENAI_API_KEY / PINECONE_API_KEY 설정 필요")

client = OpenAI(api_key=OPENAI_API_KEY)
pc = Pinecone(api_key=PINECONE_API_KEY)

INDEX_NAME = "realestate2"          # ✅ 요청한 인덱스
NAMESPACE = "__default2__"          # ✅ 기본 namespace
CSV_PATH = "data/all_chunks.csv"

index = pc.Index(INDEX_NAME)

EMBED_MODEL = "text-embedding-3-large"  # 3072-dim

# =========================
# 1) PRIORITY MAP (너 기준)
# =========================
PRIORITY_MAP = {
    "주택임대차보호법": 1,
    "시행령": 2,
    "시행규칙/대법원규칙": 3,
    "민법": 4,
    "사례/판례/피해상담": 50,
}

def get_priority(law_type: str) -> int:
    lt = (law_type or "").strip()
    if lt in PRIORITY_MAP:
        return PRIORITY_MAP[lt]
    s = re.sub(r"\s+", "", lt)
    if "주택임대차" in s:
        return 1
    if "시행령" in s:
        return 2
    if "시행규칙" in s or "대법원규칙" in s:
        return 3
    if "민법" in s:
        return 4
    if "판례" in s or "사례" in s or "피해상담" in s:
        return 50
    return 50

# =========================
# 2) TEXT UTILS
# =========================
def norm_space(s: str) -> str:
    return re.sub(r"\s+", " ", (s or "")).strip()

def split_sentences_kr(text: str) -> list[str]:
    """
    안전한 문장 분리 (너가 쓰던 방식 기반)
    """
    text = (text or "").strip()
    if not text:
        return []

    # 1) .,?,! 뒤 공백 분리
    parts = re.split(r'(?<=[\.\?\!])\s+', text)

    # 2) '다.' 기준 추가 분리(look-behind 없이)
    final = []
    for p in parts:
        sub = re.split(r'(다\.)\s*', p)
        buf = ""
        for token in sub:
            if token == "다.":
                buf += token
                if buf.strip():
                    final.append(buf.strip())
                buf = ""
            else:
                buf += token
        if buf.strip():
            final.append(buf.strip())

    final = [s.strip() for s in final if s and s.strip()]
    return final

def chunk_by_sentences(sentences: list[str], max_chars: int = 1400, overlap_sents: int = 2) -> list[str]:
    """
    문장 경계 기반 청킹(문장/조문 중간 절단 최소화)
    """
    out = []
    buf = []
    buf_len = 0

    def flush():
        nonlocal buf, buf_len
        if buf:
            out.append(norm_space(" ".join(buf)))
        buf = []
        buf_len = 0

    for s in sentences:
        s = norm_space(s)
        if not s:
            continue

        # 단일 문장이 너무 길면 최후수단으로 자름(드뭄)
        if len(s) > max_chars:
            flush()
            for i in range(0, len(s), max_chars):
                piece = norm_space(s[i:i+max_chars])
                if len(piece) >= 30:
                    out.append(piece)
            continue

        if buf_len + len(s) + 1 > max_chars:
            flush()
            # overlap 적용(이전 청크 마지막 overlap_sents 문장을 carry)
            if overlap_sents > 0 and out:
                prev_sents = split_sentences_kr(out[-1])
                carry = prev_sents[-overlap_sents:] if len(prev_sents) >= overlap_sents else prev_sents
                buf = carry[:]
                buf_len = len(" ".join(buf))

        buf.append(s)
        buf_len += len(s) + 1

    flush()
    return [c for c in out if len(c) >= 30]

# =========================
# 3) Embedding (batch)
# =========================
def embed_batch(texts: list[str]) -> list[list[float]]:
    resp = client.embeddings.create(model=EMBED_MODEL, input=texts)
    # OpenAI가 반환 순서를 입력과 동일하게 줌
    return [d.embedding for d in resp.data]

# =========================
# 4) Load CSV -> Article 복원 -> 재청킹
# =========================
df = pd.read_csv(CSV_PATH)

required = ["source", "law_type", "article", "chunk_id", "text"]
missing = [c for c in required if c not in df.columns]
if missing:
    raise ValueError(f"❌ CSV에 필요한 컬럼이 없습니다: {missing}\n현재 컬럼: {list(df.columns)}")

# chunk_id 정렬용
df["chunk_id_num"] = pd.to_numeric(df["chunk_id"], errors="coerce")
df = df.sort_values(by=["source", "law_type", "article", "chunk_id_num"], na_position="last")

# (source, law_type, article) 단위로 조문 텍스트 합치기
articles = []
for (source, law_type, article), g in df.groupby(["source", "law_type", "article"], dropna=False):
    parts = [norm_space(t) for t in g["text"].astype(str).tolist() if norm_space(t)]
    full_text = norm_space("\n".join(parts))
    if len(full_text) < 50:
        continue
    articles.append({
        "source": source,
        "law_type": law_type,
        "priority": get_priority(law_type),
        "article": article,
        "full_text": full_text,
    })

articles_df = pd.DataFrame(articles)
print("✅ 조문(article) 복원 개수:", len(articles_df))

# 재청킹하여 업서트 레코드 생성
MAX_CHARS = 1400
OVERLAP_SENTS = 2

records = []
for row in articles_df.itertuples(index=False):
    sents = split_sentences_kr(row.full_text)
    if not sents:
        continue
    chunks = chunk_by_sentences(sents, max_chars=MAX_CHARS, overlap_sents=OVERLAP_SENTS)

    for i, ch in enumerate(chunks):
        records.append({
            "id": str(uuid.uuid4()),
            "category": "law",  # 필요하면 여기서 바꿔도 됨
            "priority": int(row.priority),
            "law_type": row.law_type,
            "article": row.article,
            "chunk_id": i,
            "source": row.source,
            "text": ch,
        })

out_df = pd.DataFrame(records)
print("✅ 재청킹 결과 chunk 수:", len(out_df))
print(out_df.head(3))

# =========================
# 5) Upsert to Pinecone (realestate)
# =========================
BATCH = 64

def upsert_df(batch_df: pd.DataFrame):
    texts = batch_df["text"].tolist()
    vecs = embed_batch(texts)

    to_upsert = []
    for vec, r in zip(vecs, batch_df.to_dict("records")):
        md = {
            "category": r["category"],
            "priority": r["priority"],
            "law_type": r["law_type"],
            "article": r["article"],
            "chunk_id": r["chunk_id"],
            "source": r["source"],
            "text": r["text"],   # ✅ 발췌용 핵심
        }
        to_upsert.append((r["id"], vec, md))

    index.upsert(vectors=to_upsert, namespace=NAMESPACE)

# 실행
total = len(out_df)
for start in range(0, total, BATCH):
    batch_df = out_df.iloc[start:start+BATCH]
    upsert_df(batch_df)
    if (start // BATCH) % 10 == 0:
        print(f"  ... upsert {start + len(batch_df)}/{total}")

print("✅ 완료! 인덱스:", INDEX_NAME, "| namespace:", NAMESPACE)
print("👉 확인:", index.describe_index_stats())

✅ 조문(article) 복원 개수: 119
✅ 재청킹 결과 chunk 수: 165
                                     id category  priority    law_type  \
0  d9cab39a-c371-4c1b-9a16-a5b24bd6b212      law        50  사례/판례/피해상담   
1  73d58879-0530-4840-a732-2830bfb8982a      law        50  사례/판례/피해상담   
2  5b51377b-502a-43cd-84df-3d5a46545838      law        50  사례/판례/피해상담   

  article  chunk_id             source  \
0    사례 1         0  2025전세피해지원사례집.pdf   
1   사례 10         0  2025전세피해지원사례집.pdf   
2   사례 10         1  2025전세피해지원사례집.pdf   

                                                text  
0  전세대출 연체 문제 주요 전세피해주택에서 퇴거하고자 하나 임대인이 연락 두절(또는 ...  
1  다가구주택 임차인의 우선매수권 양도 11 우선매수권을 양도하지 못한 임차인 (LH가...  
2  • 경매가정 : LH 감정가 6.5억원, 낙찰가격 5억원, 선순위 근저당 4.5억원...  
  ... upsert 64/165
✅ 완료! 인덱스: realestate2 | namespace: __default2__
👉 확인: {'_response_info': {'raw_headers': {'connection': 'keep-alive',
                                    'content-length': '187',
                                    'content-type': 'application/js